<a href="https://colab.research.google.com/github/yehaa2004/MY-PROJECTS/blob/main/Spotify_song_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("/content/archive.zip")

In [3]:
data.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [4]:
data.drop( columns =  ["explicit","time_signature","duration_ms"  ] , inplace  = True)

In [5]:
data.drop( data.columns[0] , inplace  = True,axis=1)

In [6]:
data.head()

,track_id,artists,album_name,track_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,acoustic


## Similarity Function and Corresponding Columns

1. **Cosine Similarity**:
   - Columns: 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'
   - Explanation: Cosine similarity calculates the cosine of the angle between two vectors. These columns represent various audio features of the songs, which are commonly used to compute the similarity between songs based on their musical characteristics.



In [7]:
sampled_data = data.groupby('track_genre', group_keys=False).apply(lambda x: x.sample(min(len(x), 200)))

sampled_data.reset_index(drop=True, inplace=True)

print("Sampled Data Shape:", sampled_data.shape)
print(sampled_data.head())

Sampled Data Shape: (22800, 17)
                 track_id                                            artists  \
0  2mKjs6s0Z1imKKb6gOk628                                         Jason Mraz   
1  71nADpZkwmhx8tnnPKXHMP  Covers Culture;Acoustic Covers Culture;Lounge ...   
2  25wiOYoBrS6ITwcPGlTSXi                                   Front Porch Step   
3  1DxwWEVXzZB2IqGxI6GYRK                              Vendredi;Chrissy Dave   
4  3k6s9QkAZfENWHO3NhAQOn                                    Chris Cresswell   

                                          album_name  \
0                      Waiting for My Rocket to Come   
1             Covers - Chill Covers Of Popular Songs   
2                   I Never Loved Before I Found You   
3  Chill Out Covers - Chillout Covers - Chill Aco...   
4                                    One Week Record   

                                          track_name  popularity  \
0                                     You and I Both          57   
1  Lost Cause 

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_data = sampled_data.drop(['track_id', 'artists', 'album_name', 'track_name', 'track_genre'], axis=1)

cosine_data_normalized = (cosine_data - cosine_data.mean()) / cosine_data.std()

cosine_similarity_matrix = cosine_similarity(cosine_data_normalized)

print("Cosine Similarity Matrix:")
print(cosine_similarity_matrix)
print(len(cosine_similarity_matrix))

Cosine Similarity Matrix:
[[ 1.         -0.08727389  0.4047149  ...  0.46821387  0.0438995
   0.46560544]
 [-0.08727389  1.         -0.19464947 ... -0.02246099 -0.1835904
  -0.02275555]
 [ 0.4047149  -0.19464947  1.         ...  0.83366272  0.33059287
   0.44755156]
 ...
 [ 0.46821387 -0.02246099  0.83366272 ...  1.          0.58944683
   0.81672782]
 [ 0.0438995  -0.1835904   0.33059287 ...  0.58944683  1.
   0.54063361]
 [ 0.46560544 -0.02275555  0.44755156 ...  0.81672782  0.54063361
   1.        ]]
22800


In [9]:
cosine_data.size

273600

In [10]:
data.size

1938000

In [11]:
import numpy as np

# Function to recommend songs based on the user's playlist
def recommend_songs(input_songs):
    input_song_indices = []
    for name in input_songs:
        song_indices = sampled_data.index[sampled_data['track_name'] == name].tolist()
        if song_indices:
            input_song_indices.extend(song_indices)
        else:
            print(f"Song '{name}' not found in the dataset.")

    if not input_song_indices:
        return []

    average_similarity_scores = np.mean(cosine_similarity_matrix[input_song_indices], axis=0)
    sorted_indices = np.argsort(average_similarity_scores)[::-1]
    recommended_indices = [idx for idx in sorted_indices if idx not in input_song_indices]
    top_10_recommendations = recommended_indices[:10]
    recommended_song_names = [sampled_data.loc[idx, 'track_name'] for idx in top_10_recommendations]

    return recommended_song_names

# Input: List of song names
input_songs = []
print("Enter your playlist (one song per line):")
while True:
    song = input().strip()
    if not song:
        break
    input_songs.append(song)

# Print the input songs
print("\nInput Songs:")
print("\n".join(input_songs))

# Get recommendations based on the input songs
recommended_songs = recommend_songs(input_songs)

# Print the recommended songs
print("\nTop-10 Recommended Songs:")
for song in recommended_songs:
    print(song)



Enter your playlist (one song per line):
Bad Liar


Input Songs:
Bad Liar

Top-10 Recommended Songs:
小酒窩
一個人生活 - 幸福蛻變版
Me Apaixonei (A Primeira Vez Que Eu Te Vi)
Read All About It, Pt. III
Jesus Loves Me (Traditional Mix)
Welcome to Wonderland
女神
Free With You
Free With You
Nobody Knows
